In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

import warnings
from IPython.display import display, HTML, Javascript
display(HTML("<style>.container { width:70% !important; }</style>"))
pd.options.display.max_columns = None
warnings.filterwarnings('ignore') 
%autosave 60

Autosaving every 60 seconds


# HW1. Определение контактной информации в объявлении

- В этом задании вам предстоит реализовать решение для поиска контактов в объявлениях
- Таргет is_bad. Если is_bad = 1 - контакты в объявлении есть, иначе нет
- В train.csv разметка может быть неточная
- Задача показать результат ROC AUC >= 0.8
- ROC AUC будет усредняться по каждой категории: оценка сначала в разрезе каждой категории, а потом усреднение
- ВАЖНО: На платформе установлены следующие библиотеки (необходимо ограничиться ими):
 - joblib==1.3.2
 - numpy==1.23.5
 - pandas==2.2.2
 - scipy==1.11.4
 - scikit-learn==1.4.2
 - lightgbm==4.3.0

In [2]:
train = pd.read_csv(r"C:\Users\Windows 10\Downloads\NLP\train.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984487 entries, 0 to 984486
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               984487 non-null  object 
 1   description         984487 non-null  object 
 2   subcategory         984487 non-null  object 
 3   category            984487 non-null  object 
 4   price               926439 non-null  float64
 5   region              984487 non-null  object 
 6   city                984487 non-null  object 
 7   datetime_submitted  984487 non-null  object 
 8   is_bad              984487 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 67.6+ MB


In [4]:
train

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0
...,...,...,...,...,...,...,...,...,...
984482,Штампы 61r,+79780979761/\nДенис/\nширину обода и вылет на...,Запчасти и аксессуары,Транспорт,2000.0,Крым,Мирное,2019-10-09 23:55:54.099019,1
984483,Juul,Зарядка и под в комплекте.,Коллекционирование,Хобби и отдых,1200.0,Россия,Москва,2019-10-09 23:56:13.283059,0
984484,Белый стул IKEA,Белый стул Ikea /\nСостояние требует ремонта: ...,Мебель и интерьер,Для дома и дачи,500.0,Россия,Санкт-Петербург,2019-10-09 23:58:42.377241,0
984485,"5-к квартира, 32 м², 2/2 эт.",Бронирование с октября по январь 1010 г./\n ...,Квартиры,Недвижимость,800.0,Калининградская область,Калининград,2019-10-09 23:59:26.744680,0


In [5]:
# предобработка данных
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace('/', ' ')
    return text

In [6]:
train['description'] = train['description'].apply(preprocess_text)

In [7]:
%%time
# создание признаков
tfidf = TfidfVectorizer(max_features=10000)
X = tfidf.fit_transform(train['description'])

CPU times: total: 1min
Wall time: 1min 1s


In [10]:
print(X)

  (0, 2727)	0.12597444455001602
  (0, 4990)	0.16241459369858172
  (0, 3862)	0.10232580196254752
  (0, 2669)	0.147655040565126
  (0, 9013)	0.11849793284910472
  (0, 5625)	0.1839493123047285
  (0, 6899)	0.20518594989791245
  (0, 5336)	0.1639370094639857
  (0, 5415)	0.273475003388591
  (0, 6450)	0.20493775156964342
  (0, 8079)	0.14496081403843802
  (0, 2210)	0.21385082140600864
  (0, 3372)	0.07361270799003317
  (0, 9993)	0.22328039143973696
  (0, 5311)	0.2682732302272515
  (0, 6729)	0.2072354012784642
  (0, 2731)	0.08709316223950127
  (0, 171)	0.14656989778773843
  (0, 808)	0.20934889953812516
  (0, 5201)	0.17473307021063203
  (0, 8491)	0.23342936767851674
  (0, 7586)	0.1155826633797192
  (0, 5236)	0.21077185855258196
  (0, 9195)	0.17856005314843545
  (0, 4718)	0.17538056865323692
  :	:
  (984486, 6249)	0.09981672368628006
  (984486, 6528)	0.07565707218888
  (984486, 6189)	0.11992418429166142
  (984486, 3591)	0.1545420863871617
  (984486, 2439)	0.07336355962923756
  (984486, 8110)	0.10048

In [14]:
%%time
# cоздание признаков и обучение модели
tfidf = TfidfVectorizer(max_features=10000)
model = LGBMClassifier()

pipeline = Pipeline([
    ('tfidf', tfidf),
    ('model', model)])

X_train, X_test, y_train, y_test = train_test_split(train['description'], train['is_bad'], test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 190513, number of negative: 597076
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 13.186790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1240912
[LightGBM] [Info] Number of data points in the train set: 787589, number of used features: 10000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.241894 -> initscore=-1.142324
[LightGBM] [Info] Start training from score -1.142324
CPU times: total: 7min 15s
Wall time: 2min 28s


Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=10000)),
                ('model', LGBMClassifier())])

In [15]:
# оценка модели
y_pred = pipeline.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred)
print(f'ROC AUC: {roc_auc}')

ROC AUC: 0.9267530878162108


In [16]:
# cохранение модели и пайплайна
joblib.dump(pipeline, 'text_classification_pipeline.joblib')

['text_classification_pipeline.joblib']